In [ ]:
cd /home/mateuszg/http2vec

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
lm = "roberta"
dataset = "CSIC2010"
clf_name = "rf"
size = 3072

In [ ]:
import json
import pandas as pd
from transformers import RobertaTokenizer

from http2vec.evaluation import (
    get_data,
    get_classifier_fn,
    get_vectorizer,
    get_classifier
)

In [ ]:
data = get_data(dataset)

clf = get_classifier(lm, dataset, size, clf_name)
vectorizer = get_vectorizer(lm, dataset, size)
clf_fun = get_classifier_fn(
    vectorizer,
    f"data/tokenizers/{dataset}",
    clf,
)

tokenizer = RobertaTokenizer.from_pretrained(
    f"data/tokenizers/{dataset}"
)

# Explain single instance

In [ ]:
import shap

explainer = shap.Explainer(clf_fun, masker=tokenizer, algorithm="auto")
shap_values = explainer(data["text"][0:1])
# visualize the first prediction's explanation for the POSITIVE output class
shap.plots.text(shap_values[0, :, 0])

# Explain overall results

In [ ]:
to_explain = pd.read_csv("CSIC2010-n-closest.csv")

In [ ]:
explainer = shap.Explainer(clf_fun, masker=tokenizer, algorithm="auto")
shap_values = explainer(to_explain["text"])
shap.plots.text(shap_values[0, :, 0])

In [ ]:
a = shap_values.sum(0)
a.values = a.values[:, 0] #/ abs(a.values[:, 0].max())
a.values[a.values < 0] = 0 # We dont care for "normal" tokens

In [ ]:
shap.plots.bar(a, max_display=25)

In [ ]:
pd.DataFrame(zip(a.data, a.values)).sort_values(by=1, ascending=False)

In [ ]:
len(shap_values[0, :, 0].data)